Descobrindo a senha com DEAP
============================



## Importações



In [1]:
import random
import numpy as np

from deap import base
from deap import creator
from deap import tools
from deap.algorithms import eaSimple

from funcoes import gene_letras

## Problema da senha usando `DEAP`



Primeiro definimos as constantes.



In [2]:
# relacionadas ao problema a ser resolvido
SENHA = "correcthorsebatterystaple"
LETRAS_POSSIVEIS = "abcdefghijklmnopqrstuvwxyz"
NUM_GENES = len(SENHA)

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.15
NUM_COMBATENTES_NO_TORNEIO = 3
TAMANHO_HALL_DA_FAMA = 1

Precisamos definir a função objetivo também.



In [3]:
def funcao_objetivo_senha(individuo, senha_verdadeira):
    """Computa a funcao objetivo de um individuo no problema da senha

    Args:
      individiuo: lista contendo as letras da senha
      senha_verdadeira: a senha que você está tentando descobrir

    Returns:
      Uma tupla com a  "distância" entre a senha proposta e a senha verdadeira.
      Essa distância é medida letra por letra. Quanto mais distante uma letra
      for da que deveria ser, maior é essa distância.
    """
    diferenca = 0

    for letra_candidato, letra_oficial in zip(individuo, senha_verdadeira):
        diferenca = diferenca + abs(ord(letra_candidato) - ord(letra_oficial))

    return (diferenca, )

O `DEAP` não tem uma função de mutação de letras, precisamos usar a que nós criamos na disciplina. Tem um pequeno porém, o `DEAP` espera que a função de mutação retorne um indivíduo dentro de uma tupla. Não sei dizer o motivo dessa escolha, temos que aceitar e seguir.



In [4]:
def mutacao_senha(individuo, letras):
    """Realiza a mutação de um gene no problema da senha.

    Args:
      individuo: uma lista representado um individuo no problema da senha
      letras: letras possíveis de serem sorteadas.

    Return:
      Uma tupla de um individuo (senha) com um gene mutado.
    """
    gene = random.randint(0, len(individuo) - 1)
    individuo[gene] = gene_letras(letras)
    return (individuo, )

O restante não tem segredo. Único detalhe é que agora o problema é de minimização, então precisamos alterar isso.



In [5]:
# Devemos alterar aqui por ser um problema de minimização
creator.create("Fitness_min", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Fitness_min)

toolbox = base.Toolbox()

toolbox.register("cria_gene", gene_letras, LETRAS_POSSIVEIS)
toolbox.register(
    "individual",
    tools.initRepeat,
    creator.Individual,
    toolbox.cria_gene,
    NUM_GENES,
)
toolbox.register(
    "population", tools.initRepeat, list, toolbox.individual, TAMANHO_POP
)
toolbox.register("evaluate", funcao_objetivo_senha, SENHA)
toolbox.register(
    "select", tools.selTournament, tournsize=NUM_COMBATENTES_NO_TORNEIO
)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register(
    "mutate",
    mutacao_senha,
    letras=LETRAS_POSSIVEIS
)

hall_da_fama = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

estatisticas = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas.register("avg", np.mean)
estatisticas.register("std", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

populacao_inicial = toolbox.population()

populacao_final, log = eaSimple(
    populacao_inicial,
    toolbox,
    cxpb=CHANCE_CRUZAMENTO,
    mutpb=CHANCE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas,
    halloffame=hall_da_fama,
    verbose=True,
)

print()
print("Hall da fama:")
print("".join(hall_da_fama.items[0]))

gen	nevals	avg  	std    	min	max
0  	50    	221.1	32.2647	145	273
1  	26    	189.82	25.1712	143	234
2  	36    	165.74	18.4757	121	211
3  	38    	150.46	15.166 	121	194
4  	35    	136.4 	11.7745	111	160
5  	14    	127.14	10.0896	99 	154
6  	29    	121.02	13.1156	98 	155
7  	20    	112.08	12.2863	94 	145
8  	19    	103.7 	8.25167	94 	131
9  	25    	98.38 	5.34374	91 	115
10 	23    	94.68 	5.55496	72 	113
11 	30    	91.38 	6.72574	72 	111
12 	31    	87.44 	8.45733	71 	122
13 	30    	80.68 	6.61949	71 	94 
14 	34    	77.36 	6.11804	68 	94 
15 	34    	73.58 	4.79621	66 	87 
16 	32    	70.74 	2.90386	57 	78 
17 	38    	71.2  	6.02329	57 	90 
18 	32    	66.54 	4.59221	51 	71 
19 	38    	64.24 	6.62891	51 	83 
20 	27    	58.58 	5.69593	51 	74 
21 	31    	55.46 	4.90392	51 	69 
22 	30    	53.88 	4.99055	50 	68 
23 	32    	52.28 	4.14266	50 	73 
24 	35    	51.3  	3.66197	47 	72 
25 	31    	51.32 	3.80232	47 	70 
26 	30    	49.86 	3.05948	43 	67 
27 	24    	49.12 	3.50223	43 	64 
28 	24    	48.64